## Задание 

    Написать приложение, которое собирает основные новости с сайта на выбор lenta.ru, news.mail.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
        название источника;
        наименование новости;
        ссылку на новость;
        дата публикации.
    Сложить собранные данные в БД

Минимум один сайт, максимум - все три


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from lxml import html
from pprint import pprint

import pymongo
from pymongo import MongoClient
import datetime

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'}
now = datetime.datetime.now()

In [2]:
# Подключение к локальному серверу MongoDB.
client = MongoClient('127.0.0.1', 27017)
db = client['news']
news = db.news

In [3]:
# lenta

source_name = 'https://lenta.ru'
response = requests.get(source_name)
dom = html.fromstring(response.text)

item = dom.xpath('//time[@class = "g-time"]/../..')
for el in item:
    url = 'https://lenta.ru' + el.xpath('./a/@href')[0]
    title = el.xpath('./a/text()')[0]
    title = title.replace('\xa0', ' ')
    publication_date = el.xpath('./a/time/@title')[0]
    
    result_lenta_dic = {
        'url': url,
        'title': title, 
        'publication_date': publication_date, 
        'source_name': source_name
    }
    # добавляем в БД
    news.insert_one(result_lenta_dic)

In [4]:
# yandex

sourse_name = 'https://yandex.ru/news'
rubrics = ['business', 'personal_feed', 'society', 'world', 'sport', 'incident', 'science']
response = requests.get(sourse_name)
dom = html.fromstring(response.text)

for rubric in rubrics:
    item = dom.xpath(f"//a[contains(@href, 'rubric={rubric}') and @class = 'mg-card__link']/ancestor::article")
    for i in item:
        url = i.xpath(".//a[@class = 'mg-card__link']/@href")[0]
        title = i.xpath(".//h2[@class = 'mg-card__title']/text()")[0] 
        title = title.replace('\xa0', ' ')
        # Все новости на странице имеют текущую дату
        publication_date = now.strftime("%d-%m-%Y")
        sourse_name = i.xpath(".//a[@class = 'mg-card__source-link']/text()")[0]
        
        result_yandex_dic = {
            'url': url,
            'title': title, 
            'publication_date': publication_date, 
            'source_name': source_name
        }
        # добавляем в БД
        news.insert_one(result_yandex_dic)

In [5]:
# news mail
from datetime import datetime

url = 'https://news.mail.ru/'
responce = requests.get(url, headers=headers)
dom = html.fromstring(responce.text)

news_mail = dom.xpath("//div[@class='js-module']//li[@class='list__item' ]//@href "
                 "| //td[@class='daynews__main']//@href | //td[@class='daynews__items']//@href")

news_mail = ['https://news.mail.ru/economics/47736764/']
for url in news_mail:
    news_detail = requests.get(url, headers=headers)
    dom_news_detail = html.fromstring(news_detail.text)
    
    title = dom_news_detail.xpath(".//h1[@class='hdr__inner']//text()")[0]
    source_name = dom_news_detail.xpath("//a[@class = 'link color_gray breadcrumbs__link']/span/text()")[0]
    times = datetime.strptime(dom_news_detail.xpath(".//span[@class='note__text breadcrumbs__text js-ago']//@datetime")[0][:19], "%Y-%m-%dT%H:%M:%S")    
    
    result_mail_dic = {
        'url': url,
        'title': title, 
        'publication_date': publication_date, 
        'sourse_name': source_name
    }
    # добавляем в БД
    news.insert_one(result_mail_dic)

db.news.delete_many({})

In [6]:
print(f'Количество документов в коллекции MongoDB: \n{db.news.count_documents({})}')

Количество документов в коллекции MongoDB: 
46
